In [31]:
import pandas as pd
import numpy as np


In [32]:
#import data
college_score_path = 'data\collegescorecard.csv'
crime_path = 'data\crime_2015.csv'
dict_path = 'data\collegescorecarddatadictionary_09_12_2015.csv'

college_df = pd.read_csv(college_score_path)
crime_df = pd.read_csv(crime_path)
college_dict = pd.read_csv(dict_path)

C:\Users\Michael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (7,9,1427,1542,1561,1575,1725,1726,1727,1728,1729) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
#list(college_df). CIP11 is for Infomation Technology related degrees.

In [34]:
college_df[['unitid', 'cip11bachl']].groupby('cip11bachl').count()

college_offers = college_df[college_df.cip11bachl > 0]
college_offers.head()

,unitid,opeid,opeid6,instnm,city,stabbr,zip,accredagency,insturl,npcurl,...,nopell_rpy_3yr_rt_supp,female_rpy_3yr_rt_supp,male_rpy_3yr_rt_supp,firstgen_rpy_3yr_rt_supp,notfirstgen_rpy_3yr_rt_supp,c150_l4_pooled_supp,c150_4_pooled_supp,c200_l4_pooled_supp,c200_4_pooled_supp,location
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,galileo.aamu.edu/netpricecalculator/npcalc.htm,...,0.45251396648,0.484818805093,0.406,0.423580786026,0.442452830189,NaN,0.30871831265508,NaN,0.35168556430446,POINT(34.7834 -86.5685)
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,www.uab.edu,www.collegeportraits.org/AL/UAB/estimator/agree,...,0.795454545455,0.760521885522,0.755344418052,0.751552795031,0.765074770863,NaN,0.50854980789381,NaN,0.4730923834537,POINT(33.5022 -86.8092)
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,www.amridgeuniversity.edu,tcc.noellevitz.com/(S(miwoihs5stz5cpyifh4nczu0...,...,0.775,0.610465116279,0.686567164179,0.632653061224,0.65,NaN,PrivacySuppressed,NaN,PrivacySuppressed,POINT(32.3626 -86.174)
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu,finaid.uah.edu/,...,0.85140562249,0.768378650554,0.798440979955,0.778156996587,0.791666666667,NaN,0.47821134615384,NaN,0.49412939796716,POINT(34.7228 -86.6384)
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu/email/index.aspx,www.alasu.edu/cost-aid/forms/calculator/index....,...,0.380829015544,0.364423717522,0.286530612245,0.315737051793,0.334633385335,NaN,0.25748195545488,NaN,0.29365162671232,POINT(32.3643 -86.2957)


In [63]:
#college_dict.loc[(college_dict.iloc[:, 1].str.contains('cip', na = False)), :]

req_columns = ['unitid', 'opeid', 'opeid6', 'instnm', 'city', 'stabbr', 'zip', 'cip11bachl']
trim_col_df = college_offers[req_columns]
#trim_col_df.head()

#violent crime and property crime are the sum of two broad types of crime. 
#these numbers should provide a good summary of how the city performs in terms of crime rate
crime_df = crime_df.dropna(axis = 0, how = 'any')
crime_df_impt = crime_df.loc[:, ['violentcrime', 'propertycrime', 'state', 'city']  ]
trim_col_df = trim_col_df.merge(crime_df_impt, left_on = ['city', 'stabbr'], right_on = ['city', 'state'])



In [69]:
#determining crime rate
low_crime = (trim_col_df['violentcrime'] < crime_df['violentcrime'].mean()) & (trim_col_df['propertycrime'] < crime_df['propertycrime'].mean())

col_df_lowcrim = trim_col_df.loc[low_crime, :]

In [80]:
#placing weights on violent vs property crime. This is because value of property crime is much higher than violent crimes
#As such, the imbalance must be weighted. 
col_df_lowcrim['crimescore'] = 0.7*col_df_lowcrim['violentcrime'] + 0.3*col_df_lowcrim['propertycrime']
result = col_df_lowcrim.sort_values('crimescore', ascending = True)

C:\Users\Michael\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [83]:
#20 colleges with lowest crime rate in the country that offers the degree that Marie needs
result['instnm'].head(20)

713                       Stevens-Henager College-Logan
712                               Utah State University
437                       Northwood University-Michigan
601               Bloomsburg University of Pennsylvania
711                      Brigham Young University-Provo
726                                Mary Baldwin College
787              Minnesota School of Business-Rochester
788              National American University-Rochester
603                       Franklin and Marshall College
725                                   Lynchburg College
724                                  Liberty University
723                            James Madison University
722                        Eastern Mennonite University
731                  University of Virginia-Main Campus
778                         Rasmussen College-Wisconsin
777                   University of Wisconsin-Green Bay
779                          Globe University–Green Bay
609    Pennsylvania State University-Penn State 

0        NormalAL
1    BirminghamAL
2    MontgomeryAL
3    HuntsvilleAL
4    MontgomeryAL
Name: city_state, dtype: object